In [32]:
import requests
import os
from importlib.machinery import SourceFileLoader
import pandas as pd


try:
    config = SourceFileLoader("config", "config.py").load_module()

    bungie_api_key = config.BUNGIE_API_KEY
    endpoint = "https://www.bungie.net/Platform/Content/Rss/NewsArticles/{pageToken}/"
    page_token = "0"
    include_body = True
    headers = {
        "X-API-Key": bungie_api_key
    }
    params = {
        "includebody": include_body
    }

    results = []

    while page_token is not None:
        response = requests.get(endpoint.format(pageToken=page_token), headers=headers, params=params)

        if response.status_code == 200:
            json_response = response.json()
            if 'NewsArticles' in json_response['Response']:
                results.extend(json_response['Response']['NewsArticles'])
            else:
                print("No NewsArticles found in the response.")
                break
            page_token = json_response['Response']['NextPaginationToken']
        else:
            print("Error:", response.status_code)
            print("Response:", response.text)
            break
except Exception as e:
    print("An error occurred:", str(e))

An error occurred: 'NextPaginationToken'


In [33]:
# Desired keys
keys = ['Title', 'Link', 'PubDate', 'Description']

# Filtering the results
filtered_results = [{key: item[key] for key in keys} for item in results]

# Convert the filtered list into a DataFrame
df = pd.DataFrame(filtered_results)

In [34]:
import re
pattern = r'Update|Hotfix'
df = df[df['Title'].str.contains(pattern, regex=True, flags=re.IGNORECASE)]
df = df.iloc[0:25].reset_index()
# Print the filtered DataFrame


In [35]:
base_url = "https://www.bungie.net"

# Prepending the base URL to each 'Link' in the DataFrame
df['Link'] = df['Link'].apply(lambda x: base_url + x)

In [37]:
df.iloc[0]['Link']

'https://www.bungie.net/7/en/News/Article/update_patch_notes_7302'